In [1]:
import sys
import os

# Determine the project root directory
try:
    # This will work when running as a script
    script_dir = os.path.dirname(os.path.abspath(__file__))
    root_dir = os.path.dirname(script_dir)
except NameError:
    # This will work in interactive environments like Jupyter notebooks
    root_dir = os.path.abspath('')
    while not os.path.exists(os.path.join(root_dir, 'config.py')):
        root_dir = os.path.dirname(root_dir)
        if root_dir == os.path.dirname(root_dir):  # reached the system root directory
            raise FileNotFoundError("Could not find the project root directory containing config.py")

# Add the root directory to the Python path
sys.path.append(root_dir)

# Now you can import from config
from config import DB_HOST, DB_NAME, DB_USER, DB_PASSWORD

# Define paths relative to the root directory
data_dir = os.path.join(root_dir, 'data')
raw_data_dir = os.path.join(root_dir, 'data', 'raw')
output_dir = os.path.join(root_dir, 'data', 'output')
vis_dir = os.path.join(root_dir, 'visualisations')
dashboard_dir = os.path.join(root_dir, 'dashboard')

# Ensure directories exist
os.makedirs(data_dir, exist_ok=True)
os.makedirs(raw_data_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(vis_dir, exist_ok=True)
os.makedirs(dashboard_dir, exist_ok=True)

# Print paths for verification
print(f"Project root: {root_dir}")
print(f"Raw data directory: {raw_data_dir}")
print(f"Output directory: {output_dir}")
print(f"Visualisations directory: {vis_dir}")
print(f"Dashboard directory: {dashboard_dir}")

import requests
import numpy as np
import zipfile
import io
import pandas as pd
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.schema import CreateSchema
from sqlalchemy.exc import ProgrammingError, OperationalError
from sqlalchemy.exc import ProgrammingError
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse
from urllib.parse import urljoin
from zipfile import ZipFile
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil import parser
from config import DB_HOST, DB_NAME, DB_USER, DB_PASSWORD

Project root: c:\Users\willi\OneDrive\Documents\NHS Digital Project\nhs-digital-project
Raw data directory: c:\Users\willi\OneDrive\Documents\NHS Digital Project\nhs-digital-project\data\raw
Output directory: c:\Users\willi\OneDrive\Documents\NHS Digital Project\nhs-digital-project\data\output
Visualisations directory: c:\Users\willi\OneDrive\Documents\NHS Digital Project\nhs-digital-project\visualisations
Dashboard directory: c:\Users\willi\OneDrive\Documents\NHS Digital Project\nhs-digital-project\dashboard


In [2]:
# Database connection parameters
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
engine = create_engine(connection_string)

# Test the connection
try:
    with engine.connect() as conn:
        print("Successfully connected to the database!")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully connected to the database!


In [3]:
# URL of the page containing the table
url = "https://www.sbs.nhs.uk/supplier-information/ccg-icb-list/"

# Send a GET request to fetch the HTML content
response = requests.get(url)
html_content = response.content

# Parse HTML using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table with class 'd1general' (assuming there is only one such table)
table = soup.find('table', class_='d1general')

# Extract column headers
headers = []
for th in table.find('thead').find_all('th'):
    headers.append(th.text.strip())

# Extract table rows
data = []
for row in table.find('tbody').find_all('tr'):
    row_data = [td.text.strip() if td.text.strip() != '' else None for td in row.find_all('td')]
    data.append(row_data)

# Create DataFrame
df_ccg_icb = pd.DataFrame(data, columns=headers)

# Replace 'None' values with the previous non-empty entity
df_ccg_icb['ICB Entity Code'] = df_ccg_icb['ICB Entity Code'].fillna(method='ffill')
df_ccg_icb['ICB New Name'] = df_ccg_icb['ICB New Name'].fillna(method='ffill')

# Display the DataFrame
print(df_ccg_icb)

    CCG Entity Code                                              CCG’S  \
0               99C                             NHS North Tyneside CCG   
1               00L                             NHS Northumberland CCG   
2               13T  NHS Newcastle Gateshead CCG (formed by merger ...   
3               01H                              NHS North Cumbria CCG   
4               00P                                 NHS Sunderland CCG   
..              ...                                                ...   
102             03Q                               NHS Vale of York CCG   
103             42D  NHS North Yorkshire CCG (formed by merger in 2...   
104             18C           NHS Herefordshire and Worcestershire CCG   
105             Y04                          NHS North East London CCG   
106             72Q                          NHS South East London CCG   

    ICB Entity Code                                       ICB New Name  
0               QHM  NHS North East an

C:\Users\willi\AppData\Local\Temp\ipykernel_17632\3225009203.py:29: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ccg_icb['ICB Entity Code'] = df_ccg_icb['ICB Entity Code'].fillna(method='ffill')
C:\Users\willi\AppData\Local\Temp\ipykernel_17632\3225009203.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ccg_icb['ICB New Name'] = df_ccg_icb['ICB New Name'].fillna(method='ffill')


In [4]:
df_ccg_icb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CCG Entity Code  107 non-null    object
 1   CCG’S            107 non-null    object
 2   ICB Entity Code  107 non-null    object
 3   ICB New Name     107 non-null    object
dtypes: object(4)
memory usage: 3.5+ KB


In [5]:
# Rename the columns to match the new naming convention
df_ccg_icb = df_ccg_icb.rename(columns={
    'CCG’S': 'CCG',
})
df_ccg_icb

,CCG Entity Code,CCG,ICB Entity Code,ICB New Name
0,99C,NHS North Tyneside CCG,QHM,NHS North East and North Cumbria Integrated Ca...
1,00L,NHS Northumberland CCG,QHM,NHS North East and North Cumbria Integrated Ca...
2,13T,NHS Newcastle Gateshead CCG (formed by merger ...,QHM,NHS North East and North Cumbria Integrated Ca...
3,01H,NHS North Cumbria CCG,QHM,NHS North East and North Cumbria Integrated Ca...
4,00P,NHS Sunderland CCG,QHM,NHS North East and North Cumbria Integrated Ca...
...,...,...,...,...
102,03Q,NHS Vale of York CCG,QOQ,NHS Humber and North Yorkshire Integrated Care...
103,42D,NHS North Yorkshire CCG (formed by merger in 2...,QOQ,NHS Humber and North Yorkshire Integrated Care...
104,18C,NHS Herefordshire and Worcestershire CCG,QGH,NHS Herefordshire and Worcestershire Integrate...
105,Y04,NHS North East London CCG,QMF,NHS North East London Integrated Care Board


In [8]:
def extract_specific_zip_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        zip_links = {
            'regional': None,
            'actual_duration': None
        }
        
        all_zip_links = soup.find_all('a', href=re.compile(r'\.zip$'))
        
        for link in all_zip_links:
            href = link['href']
            text = link.text.lower()
            
            if 'regional' in href.lower() or 'regional' in text:
                zip_links['regional'] = href
            
            if 'actual_duration' in href.lower() or 'actual duration' in text:
                zip_links['actual_duration'] = href
        
        return url, zip_links
    except requests.RequestException as e:
        return url, None

def process_urls(urls):
    all_zip_links = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {executor.submit(extract_specific_zip_links, url): url for url in urls}
        for future in as_completed(future_to_url):
            url, zip_links = future.result()
            if zip_links:
                all_zip_links[url] = zip_links
    
    return all_zip_links


def download_zip(url, download_dir):
    os.makedirs(download_dir, exist_ok=True)
    
    filename = os.path.basename(urlparse(url).path)
    file_path = os.path.join(download_dir, filename)
    
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    with open(file_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    return file_path


def process_zip_file(zip_path):
    data_frames = []
    with ZipFile(zip_path, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.endswith('.csv'):
                with zip_ref.open(file) as f:
                    df = pd.read_csv(f)
                    data_frames.append(df)
    if data_frames:
        return pd.concat(data_frames, ignore_index=True)
    return pd.DataFrame()

def process_all_zip_files(all_zip_links, download_dir):
    processed_data = {}
    for url, zip_links in all_zip_links.items():
        processed_data[url] = {}
        for data_type, link in zip_links.items():
            if link:
                try:
                    zip_path = download_zip(link, download_dir)
                    df = process_zip_file(zip_path)
                    if not df.empty:
                        processed_data[url][data_type] = df
                except Exception as e:
                    print(f"Failed to process {link}: {e}")
                    continue
    return processed_data

# List of URLs
urls = [
    'https://digital.nhs.uk/data-and-information/publications/statistical/appointments-in-general-practice/may-2024',
    'https://digital.nhs.uk/data-and-information/publications/statistical/appointments-in-general-practice/september-2021',
]       

print("Processing URLs...")
all_zip_links = process_urls(urls)
print(f"Found zip links: {all_zip_links}")

print(f"\nProcessing zip files in directory: {raw_data_dir}")
processed_data = process_all_zip_files(all_zip_links, raw_data_dir)

print("\nProcessed data structure:")
for url, data in processed_data.items():
    print(f"\nURL: {url}")
    for data_type, df in data.items():
        print(f"  {data_type}: Shape {df.shape}, Columns: {df.columns.tolist()}")

# Extract regional and actual duration data from processed_data
all_regional_data = pd.DataFrame()
all_actual_duration_data = pd.DataFrame()

print("\nExtracting and combining data...")
for url, data in processed_data.items():
    if 'regional' in data:
        print(f"Adding regional data from {url}")
        all_regional_data = pd.concat([all_regional_data, data['regional']], ignore_index=True)
    if 'actual_duration' in data:
        print(f"Adding actual duration data from {url}")
        all_actual_duration_data = pd.concat([all_actual_duration_data, data['actual_duration']], ignore_index=True)

print("\nFinal dataframes:")
print(f"all_regional_data: Shape {all_regional_data.shape}, Columns: {all_regional_data.columns.tolist()}")
print(f"all_actual_duration_data: Shape {all_actual_duration_data.shape}, Columns: {all_actual_duration_data.columns.tolist()}")

# Sample data preview
print("\nSample of all_regional_data:")
print(all_regional_data.head())
print("\nSample of all_actual_duration_data:")
print(all_actual_duration_data.head())

# Additional checks
print("\nChecking for empty dataframes:")
if all_regional_data.empty:
    print("Warning: all_regional_data is empty")
if all_actual_duration_data.empty:
    print("Warning: all_actual_duration_data is empty")

print("\nChecking for expected columns:")
expected_columns = ['CCG_NAME', 'ICB_NAME']  # Add other expected column names
for col in expected_columns:
    if col not in all_regional_data.columns:
        print(f"Warning: Expected column '{col}' not found in all_regional_data")
    if col not in all_actual_duration_data.columns:
        print(f"Warning: Expected column '{col}' not found in all_actual_duration_data")

Processing URLs...
Found zip links: {'https://digital.nhs.uk/data-and-information/publications/statistical/appointments-in-general-practice/may-2024': {'regional': 'https://files.digital.nhs.uk/5C/29B9D7/Appointments_GP_Regional_CSV_May_24.zip', 'actual_duration': 'https://files.digital.nhs.uk/17/B3D945/Appointments_GP_Actual_Duration_CSV_May_24.zip'}, 'https://digital.nhs.uk/data-and-information/publications/statistical/appointments-in-general-practice/september-2021': {'regional': 'https://files.digital.nhs.uk/E7/44A568/Appointments_GP_Regional_Sep21.zip', 'actual_duration': None}}

Processing zip files in directory: c:\Users\willi\OneDrive\Documents\NHS Digital Project\nhs-digital-project\data\raw

Processed data structure:

URL: https://digital.nhs.uk/data-and-information/publications/statistical/appointments-in-general-practice/may-2024
  regional: Shape (642124, 13), Columns: ['SUB_ICB_LOCATION_CODE', 'SUB_ICB_LOCATION_ONS_CODE', 'SUB_ICB_LOCATION_NAME', 'ICB_ONS_CODE', 'ICB_NAME

In [9]:
all_regional_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268228 entries, 0 to 1268227
Data columns (total 19 columns):
 #   Column                      Non-Null Count    Dtype 
---  ------                      --------------    ----- 
 0   SUB_ICB_LOCATION_CODE       642124 non-null   object
 1   SUB_ICB_LOCATION_ONS_CODE   642124 non-null   object
 2   SUB_ICB_LOCATION_NAME       642124 non-null   object
 3   ICB_ONS_CODE                642124 non-null   object
 4   ICB_NAME                    642124 non-null   object
 5   REGION_ONS_CODE             1268228 non-null  object
 6   REGION_NAME                 1268228 non-null  object
 7   APPOINTMENT_MONTH           642124 non-null   object
 8   APPT_STATUS                 1268228 non-null  object
 9   HCP_TYPE                    1268228 non-null  object
 10  APPT_MODE                   1268228 non-null  object
 11  TIME_BETWEEN_BOOK_AND_APPT  1268228 non-null  object
 12  COUNT_OF_APPOINTMENTS       1268228 non-null  int64 
 13  CCG_CODE    

In [10]:
# Check for any remaining unmatched CCGs
unmatched_ccgs = set(all_regional_data['CCG_NAME'].unique()) - set(df_ccg_icb['CCG'])
print("\nUnmatched CCGs in regional data:")
print(unmatched_ccgs)

if 'CCG_NAME' in all_actual_duration_data.columns:
    unmatched_ccgs = set(all_actual_duration_data['CCG_NAME'].unique()) - set(df_ccg_icb['CCG'])
    print("\nUnmatched CCGs in actual duration data:")
    print(unmatched_ccgs)
else:
    print("\nWarning: 'CCG_NAME' column not found in actual duration data. Unable to check for unmatched CCGs.")


Unmatched CCGs in regional data:
{'NHS Stoke on Trent CCG', 'NHS Mid Essex CCG', 'NHS Castle Point and Rochford CCG', 'NHS Herefordshire CCG', 'NHS Kirklees CCG', 'NHS East Sussex CCG', 'NHS Bedfordshire, Luton and Milton Keynes CCG', 'NHS West Sussex CCG', nan, 'NHS Newcastle Gateshead CCG', 'NHS County Durham CCG', 'NHS South East Staffordshire and Seisdon Peninsula CCG', 'NHS North Yorkshire CCG', 'NHS Tees Valley CCG', 'NHS Manchester CCG', 'NHS Cheshire CCG', 'NHS Shropshire, Telford and Wrekin CCG'}



In [11]:
def normalize_ccg_name(name):
    """Normalize CCG name by removing special characters and spaces."""
    return re.sub(r'[^a-zA-Z0-9]', '', name.lower())

def flexible_ccg_name_mapping(df, mapping_dict, column_name):
    """Apply CCG name mapping using flexible matching."""
    if column_name not in df.columns:
        print(f"Warning: '{column_name}' not found in DataFrame. Skipping mapping.")
        return df
    
    normalized_mapping = {normalize_ccg_name(k): v for k, v in mapping_dict.items()}
    
    def map_ccg(ccg_name):
        if pd.isna(ccg_name):
            return ccg_name
        normalized_name = normalize_ccg_name(ccg_name)
        return normalized_mapping.get(normalized_name, ccg_name)
    
    df[column_name] = df[column_name].apply(map_ccg)
    return df


def apply_icb_name_mapping(df):
    icb_name_mapping = {
        'NHS Cambridgeshire and Peterborough\xa0ICB': 'NHS Cambridgeshire and Peterborough ICB',
        'NHS Norfolk and Waveney\xa0ICB': 'NHS Norfolk and Waveney ICB',
        'NHS Hampshire and Isle of Wight ICB': 'NHS Hampshire and the Isle of Wight ICB',
        'NHS Leicester, Leicestershire and Rutland ICB': 'NHS Leicester Leicestershire and Rutland ICB',
        'NHS Cornwall and the Isles of Scilly ICB': 'NHS Cornwall and The Isles of Scilly ICB',
        'NHS Buckinghamshire, Oxfordshire and Berkshire West ICB': 'NHS Buckinghamshire Oxfordshire and Berkshire West ICB',
        'NHS Bristol, North Somerset and South Gloucestershire ICB': 'NHS Bristol North Somerset and South Gloucestershire ICB',
        'NHS Bath and North East Somerset, Swindon and Wiltshire ICB': 'NHS Bath and North East Somerset Swindon and Wiltshire ICB',
        'NHS Bedfordshire, Luton and Milton Keynes ICB': 'NHS Bedfordshire Luton and Milton Keynes ICB',
        'NHS Shropshire, Telford and Wrekin ICB': 'NHS Shropshire Telford and Wrekin ICB',
        'NHS Cornwall and The Isles Of Scilly ICB' : 'NHS Cornwall and The Isles of Scilly ICB',
        'NHS Hampshire and Isle Of Wight ICB': 'NHS Hampshire and Isle of Wight ICB',
        'NHS Staffordshire and Stoke': 'NHS Staffordshire and Stoke-on-Trent ICB'
    }
    
    if 'ICB_NAME' in df.columns:
        df['ICB_NAME'] = df['ICB_NAME'].replace(icb_name_mapping)
    if 'ICB New Name' in df.columns:
        df['ICB New Name'] = df['ICB New Name'].replace(icb_name_mapping)
    return df

# Combine all regional data
regional_dfs = [data_dict['regional'] for data_dict in processed_data.values() if 'regional' in data_dict and not data_dict['regional'].empty]
if regional_dfs:
    all_regional_data = pd.concat(regional_dfs, ignore_index=True)
    print("Combined regional data shape:", all_regional_data.shape)
    
    # Drop specified columns from all_regional_data
    columns_to_drop = ['SUB_ICB_LOCATION_CODE', 'SUB_ICB_LOCATION_ONS_CODE', 'CCG_CODE', 'CCG_ONS_CODE', 'STP_ONS', 'STPNM']
    all_regional_data = all_regional_data.drop(columns=[col for col in columns_to_drop if col in all_regional_data.columns])
    
    print("Columns after dropping from regional data:", all_regional_data.columns)
else:
    print("No regional data to combine")
    all_regional_data = pd.DataFrame()

# Combine all actual duration data
actual_duration_dfs = [data_dict['actual_duration'] for data_dict in processed_data.values() if 'actual_duration' in data_dict and not data_dict['actual_duration'].empty]
if actual_duration_dfs:
    all_actual_duration_data = pd.concat(actual_duration_dfs, ignore_index=True)
    print("Combined actual duration data shape:", all_actual_duration_data.shape)
    
    # Drop specified columns from all_actual_duration_data
    columns_to_drop = ['SUB_ICB_LOCATION_CODE', 'SUB_ICB_LOCATION_ONS_CODE', 'CCG_CODE', 'CCG_ONS_CODE', 'STP_ONS_CODE']
    all_actual_duration_data = all_actual_duration_data.drop(columns=[col for col in columns_to_drop if col in all_actual_duration_data.columns])
    
    print("Columns after dropping from actual duration data:", all_actual_duration_data.columns)
else:
    print("No actual duration data to combine")
    all_actual_duration_data = pd.DataFrame()

# Merge APPOINTMENT_MONTH and Appointment_Month columns
if 'APPOINTMENT_MONTH' in all_regional_data.columns and 'Appointment_Month' in all_regional_data.columns:
    all_regional_data['APPOINTMENT_MONTH'].fillna(all_regional_data['Appointment_Month'], inplace=True)
    all_regional_data.drop(columns=['Appointment_Month'], inplace=True)

if 'APPOINTMENT_MONTH' in all_actual_duration_data.columns and 'Appointment_Month' in all_actual_duration_data.columns:
    all_actual_duration_data['APPOINTMENT_MONTH'].fillna(all_actual_duration_data['Appointment_Month'], inplace=True)
    all_actual_duration_data.drop(columns=['Appointment_Month'], inplace=True)

# CCG name mapping
ccg_name_mapping = {
    'NHS Barking and Dagenham CCG': 'NHS Barking & Dagenham CCG',
    'NHS Bath and North East Somerset Swindon and Wiltshire CCG': 'NHS Bath and North East Somerset, Swindon and Wiltshire CCG',
    'NHS Bedfordshire CCG': 'NHS Bedfordshire, Luton & Milton Keynes CCG',
    'NHS Bedfordshire Luton and Milton Keynes CCG': 'NHS Bedfordshire, Luton & Milton Keynes CCG',
    'NHS Bedfordshire, Luton and Milton Keynes CCG': 'NHS Bedfordshire, Luton & Milton Keynes CCG',
    'NHS Brighton & Hove CCG': 'NHS Brighton and Hove CCG',
    'NHS Bristol North Somerset and South Gloucestershire CCG': 'NHS Bristol, North Somerset and South Gloucestershire CCG',
    'NHS Castle Point Rayleigh and Rochford CCG': 'NHS Castlepoint and Rochford CCG',
    'NHS Castle Point and Rochford CCG': 'NHS Castlepoint and Rochford CCG',
    'NHS Cheshire CCG (formed by merger in 2020)': 'NHS Cheshire CCG',
    'NHS County Durham CCG (formed by merger in 2020)': 'NHS County Durham CCG',
    'NHS Coventry and Rugby CCG': 'NHS Coventry and Warwickshire CCG',
    'NHS Dudley CCG': 'NHS Birmingham and Solihull CCG',
    'NHS East Lancashire CCG': 'NHS East Leicestershire and Rutland CCG',
    'NHS East Sussex CCG (formed by merger in 2020)': 'NHS East Sussex CCG',
    'NHS Fylde & Wyre CCG': 'NHS Fylde and Wyre CCG',
    'NHS Hampshire Southampton and Isle of Wight CCG': 'NHS Hampshire, Southampton and Isle of Wight CCG',
    'NHS Herefordshire CCG': 'NHS Herefordshire and Worcestershire CCG',
    'NHS Herts Valleys CCG': 'NHS Herts Valley CCG',
    'NHS Heywood Middleton & Rochdale CCG': 'NHS Heywood, Middleton and Rochdale CCG',
    'NHS Heywood Middleton and Rochdale CCG': 'NHS Heywood, Middleton and Rochdale CCG',
    'NHS Kirklees CCG (formed by merger in 2021)': 'NHS Kirklees CCG',
    'NHS Manchester CCG (formed by merger in 2017; part of Manchester Health and Care Commissioning)': 'NHS Manchester CCG',
    'NHS Mid Essex CCG': 'NHS Mid-Essex CCG',
    'NHS Newcastle Gateshead CCG (formed by merger in 2015)': 'NHS Newcastle Gateshead CCG',
    'NHS North Yorkshire CCG (formed by merger in 2020)': 'NHS North Yorkshire CCG',
    'NHS Shropshire CCG': 'NHS Shropshire Telford and Wrekin CCG',
    'NHS Shropshire and Telford and Wrekin CCG': 'NHS Shropshire Telford and Wrekin CCG',
    'NHS Shropshire, Telford and Wrekin CCG': 'NHS Shropshire Telford and Wrekin CCG',
    'NHS Telford & Wrekin CCG': 'NHS Shropshire Telford and Wrekin CCG',
    'NHS Telford and Wrekin CCG': 'NHS Shropshire Telford and Wrekin CCG',
    'NHS South East Staffordshire and Seisdon Peninsula CCG': 'NHS South East Staffordshire and Eastern Peninsula CCG',
    'NHS South East Staffs and Seisdon and Peninsular CCG': 'NHS South East Staffordshire and Eastern Peninsula CCG',
    'NHS Stoke on Trent CCG': 'NHS Stoke-on-Trent CCG',
    'NHS Tees Valley CCG (formed by merger in 2020)': 'NHS Tees Valley CCG',
    'NHS West Sussex CCG (formed by merger in 2020)': 'NHS West Sussex CCG',
    'Shropshire And Telford And Wrekin STP': 'NHS Shropshire Telford and Wrekin CCG',     
    'NHS West London (Kensington and Chelsea Queens Park and Paddington)': 'NHS West London CCG',
    "NHS West London (Kensington and Chelsea Queen's Park and Paddington)": 'NHS West London CCG',
    'NHS West London (Kensington and Chelsea Queen\x92s Park and Paddington)': 'NHS West London CCG',
    "NHS West London (Kensington and Chelsea QueenÂ's Park and Paddington)": 'NHS West London CCG',
    "NHS West London (Kensington and Chelsea QueenÂ’s Park and Paddington)": 'NHS West London CCG',
}

# Apply mappings to all dataframes
print("Applying mappings to regional data...")
all_regional_data = update_icb_names(all_regional_data)
all_regional_data = apply_icb_name_mapping(all_regional_data)
all_regional_data = flexible_ccg_name_mapping(all_regional_data, ccg_name_mapping, 'CCG_NAME')

print("Applying mappings to actual duration data...")
all_actual_duration_data = update_icb_names(all_actual_duration_data)
all_actual_duration_data = apply_icb_name_mapping(all_actual_duration_data)
print("Mappings applied to actual duration data.")


print("Applying mappings to CCG-ICB mapping data...")
df_ccg_icb = update_icb_names(df_ccg_icb)
df_ccg_icb = apply_icb_name_mapping(df_ccg_icb)
df_ccg_icb = flexible_ccg_name_mapping(df_ccg_icb, ccg_name_mapping, 'CCG')

# Check for any remaining unmatched CCGs in regional data
unmatched_ccgs = set(all_regional_data['CCG_NAME'].unique()) - set(df_ccg_icb['CCG'])
print("\nUnmatched CCGs in regional data:")
print(unmatched_ccgs)

# For actual duration data, we'll check unmatched ICBs instead
unmatched_icbs = set(all_actual_duration_data['SUB_ICB_LOCATION_NAME'].unique()) - set(df_ccg_icb['ICB New Name'])
print("\nUnmatched ICBs in actual duration data:")
print(unmatched_icbs)

# New CCG to ICB mappings
new_ccg_icb_mappings = {
    'NHS Brent CCG': 'NHS North Central London ICB',
    'NHS City and Hackney CCG': 'NHS North Central London ICB',
    'NHS Barking & Dagenham CCG': 'NHS North East London ICB',
    'NHS Central London (Westminster) CCG': 'NHS North Central London ICB',
    'NHS Cumbria CCG': 'NHS North East and North Cumbria ICB',
    'NHS Ealing CCG': 'NHS North West London ICB',
    'NHS East Berkshire CCG': 'NHS Buckinghamshire Oxfordshire and Berkshire West ICB',
    'NHS Fareham and Gosport CCG': 'NHS Hampshire and the Isle of Wight ICB',
    'NHS Greater Huddersfield CCG': 'NHS West Yorkshire ICB',
    'NHS Hammersmith and Fulham CCG': 'NHS South West London ICB',
    'NHS Harrow CCG': 'NHS North West London ICB',
    'NHS Havering CCG': 'NHS North East London ICB',
    'NHS Hillingdon CCG': 'NHS North West London ICB',
    'NHS Hounslow CCG': 'NHS South West London ICB',
    'NHS Isle of Wight CCG': 'NHS Hampshire and the Isle of Wight ICB',
    'NHS Lancashire North CCG': 'NHS Lancashire and South Cumbria ICB',
    'NHS Luton CCG': 'NHS Bedfordshire Luton and Milton Keynes ICB',
    'NHS Milton Keynes CCG': 'NHS Bedfordshire Luton and Milton Keynes ICB',
    'NHS Newham CCG': 'NHS North East London ICB',
    'NHS North East Hampshire and Farnham CCG': 'NHS Hampshire and the Isle of Wight ICB',
    'NHS North Hampshire CCG': 'NHS Hampshire and the Isle of Wight ICB',
    'NHS North Kirklees CCG': 'NHS West Yorkshire ICB',
    'NHS Redbridge CCG': 'NHS North West London ICB',
    'NHS Sandwell and West Birmingham CCG': 'NHS Birmingham and Solihull ICB',
    'NHS South Eastern Hampshire CCG': 'NHS Hampshire and the Isle of Wight ICB',
    'NHS South Warwickshire CCG': 'NHS Coventry and Warwickshire ICB',
    'NHS Southampton CCG': 'NHS Hampshire and the Isle of Wight ICB',
    'NHS Surrey Heath CCG': 'NHS Surrey Heartlands ICB',
    'NHS Tower Hamlets CCG': 'NHS North East London ICB',
    'NHS Walsall CCG': 'NHS Coventry and Warwickshire ICB',
    'NHS Waltham Forest CCG': 'NHS North East London ICB',
    'NHS Warwickshire North CCG': 'NHS Coventry and Warwickshire ICB',
    'NHS West Essex CCG': 'NHS Hertfordshire and West Essex ICB',
    'NHS West Hampshire CCG': 'NHS Hampshire and the Isle of Wight ICB',
    'NHS West London (Kensington and Chelsea Queens Park and Paddington)': 'NHS North West London ICB',
    'NHS West London CCG': 'NHS North West London ICB',
    'NHS Wolverhampton CCG': 'NHS Staffordshire and Stoke-on-Trent ICB',
}


new_mappings = pd.DataFrame([
    {'CCG': ccg, 'ICB New Name': icb} for ccg, icb in new_ccg_icb_mappings.items()
])

df_ccg_icb = pd.concat([df_ccg_icb, new_mappings], ignore_index=True)

# Remove duplicates if any
df_ccg_icb = df_ccg_icb.drop_duplicates(subset=['CCG'], keep='last')

# Display the first few rows of the updated DataFrames
print("Regional data sample:")
print(all_regional_data.head())

print("Actual duration data sample:")
print(all_actual_duration_data.head())

print("CCG to ICB mapping sample:")
print(df_ccg_icb.head())

Combined regional data shape: (1268228, 19)
Columns after dropping from regional data: Index(['SUB_ICB_LOCATION_NAME', 'ICB_ONS_CODE', 'ICB_NAME', 'REGION_ONS_CODE',
       'REGION_NAME', 'APPOINTMENT_MONTH', 'APPT_STATUS', 'HCP_TYPE',
       'APPT_MODE', 'TIME_BETWEEN_BOOK_AND_APPT', 'COUNT_OF_APPOINTMENTS',
       'CCG_NAME', 'Appointment_Month'],
      dtype='object')
Combined actual duration data shape: (594021, 8)
Columns after dropping from actual duration data: Index(['SUB_ICB_LOCATION_NAME', 'ICB_ONS_CODE', 'REGION_ONS_CODE',
       'Appointment_Date', 'ACTUAL_DURATION', 'COUNT_OF_APPOINTMENTS'],
      dtype='object')
Applying mappings to regional data...


C:\Users\willi\AppData\Local\Temp\ipykernel_17632\3413477575.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_regional_data['APPOINTMENT_MONTH'].fillna(all_regional_data['Appointment_Month'], inplace=True)


NameError: name 'update_icb_names' is not defined

In [55]:
# Function to download a file
def download_file(url, save_dir):
    local_filename = os.path.join(save_dir, url.split('/')[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"Downloaded: {local_filename}")
    return local_filename

# Function to find and download XLSX files
def find_and_download_xlsx(urls, save_dir):
    downloaded_files = []
    for url in urls:
        print(f"Processing URL: {url}")
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for bad status codes
            soup = BeautifulSoup(response.content, 'html.parser')
            links = soup.find_all('a', href=True)
            
            file_found = False
            for link in links:
                href = link['href']
                if 'regional' in href.lower() and 'table' in href.lower() and (href.endswith('.xlsm') or href.endswith('.xlsx')):
                    file_url = urljoin(url, href)
                    try:
                        file_path = download_file(file_url, save_dir)
                        downloaded_files.append(file_path)
                        file_found = True
                        break  # Move to the next URL after finding and downloading a file
                    except Exception as e:
                        print(f"Error downloading {file_url}: {str(e)}")
            
            if not file_found:
                print(f"No suitable file found for URL: {url}")
        except Exception as e:
            print(f"Error processing URL {url}: {str(e)}")
    
    return downloaded_files

def process_FTE_url(file_path, sheet_name='1', skip_rows=9):
    print(f"Processing FTE file: {file_path}")
    df_xlsm = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl', skiprows=skip_rows)
    print(f"Excel file shape: {df_xlsm.shape}")
    
    # Print column names for debugging
    print(f"Columns: {df_xlsm.columns.tolist()}")
    
    # Adjust column selection based on actual column positions
    ons_code_col = df_xlsm.columns[0]
    ons_name_col = df_xlsm.columns[1]
    fte_0_4_col = df_xlsm.columns[2]
    fte_0_4_to_1_col = df_xlsm.columns[4]
    fte_1_col = df_xlsm.columns[6]
    
    fte_data = {}
    
    for _, row in df_xlsm.iterrows():
        ons_code = row[ons_code_col]
        ons_name = row[ons_name_col]
        fte_0_4 = pd.to_numeric(row[fte_0_4_col], errors='coerce')
        fte_0_4_to_1 = pd.to_numeric(row[fte_0_4_to_1_col], errors='coerce')
        fte_1 = pd.to_numeric(row[fte_1_col], errors='coerce')
        
        if pd.notna(ons_code) and pd.notna(ons_name) and re.search(r'\d', ons_name):
            fte_data[ons_code] = {
                'ONS Name': ons_name,
                '<= 0.4 FTE': fte_0_4,
                '> 0.4 to < 1 FTE': fte_0_4_to_1,
                '>= 1 FTE': fte_1
            }
    
    fte_df = pd.DataFrame.from_dict(fte_data, orient='index')
    fte_df.index.name = 'ONS Code'
    fte_df.reset_index(inplace=True)
    
    return fte_df

def process_national_categories(zip_url, download_dir):
    zip_path = download_zip(zip_url, download_dir)
    df_list = []
    with zipfile.ZipFile(zip_path, 'r') as z:
        for filename in z.namelist():
            if filename.endswith('.csv'):
                with z.open(filename) as f:
                    df = pd.read_csv(io.BytesIO(f.read()))
                    print(f"Processed file: {filename}, shape: {df.shape}")
                    df_list.append(df)
    
    if df_list:
        result = pd.concat(df_list, ignore_index=True)
        print(f"Combined national categories data shape: {result.shape}")
        return result
    else:
        print(f"No CSV files found in {zip_path}")
        return None

# Main execution
download_dir = 'C:/Users/willi/OneDrive/Documents/NHS Digital Project/Source Data'
os.makedirs(download_dir, exist_ok=True)

# Process national categories
national_categories_url = 'https://files.digital.nhs.uk/70/C5C3DD/Appointments_GP_National_Categories_CSV_May_24.zip'
national_categories_data = process_national_categories(national_categories_url, download_dir)


# Process FTE data
fte_urls = [
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-march-2024',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-april-2024',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/29-february-2024',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-january-2024',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-december-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-november-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-october-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-september-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-august-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-july-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-june-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-may-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-april-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-march-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/28-february-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-january-2023',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-december-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-november-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-october-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-september-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-august-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-july-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-june-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-may-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-april-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-march-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/28-february-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-january-2022',
    'https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-december-2021',


]

# Download the files
downloaded_files = find_and_download_xlsx(fte_urls, download_dir)

# Process the downloaded files
fte_dfs = []
for file_path in downloaded_files:
    fte_df = process_FTE_url(file_path)
    if fte_df is not None:
        fte_df['file'] = os.path.basename(file_path)
        fte_dfs.append(fte_df)

if fte_dfs:
    combined_fte_df = pd.concat(fte_dfs, ignore_index=True)
    print("\nCombined FTE DataFrame:")
    print(combined_fte_df)
else:
    print("No valid FTE data found.")

print("\nNational Categories Data:")
if isinstance(national_categories_data, pd.DataFrame):
    print(f"  Shape: {national_categories_data.shape}")
    print(f"  Columns: {national_categories_data.columns.tolist()}")
else:
    print("  Not a valid DataFrame")

Processed file: National_Categories_CSV_2024.csv, shape: (446261, 10)
Processed file: National_Categories_CSV_2023.csv, shape: (1037736, 10)
Processed file: National_Categories_CSV_2022.csv, shape: (974583, 10)
Combined national categories data shape: (2458580, 10)
Processing URL: https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/31-march-2024
Downloaded: C:/Users/willi/OneDrive/Documents/NHS Digital Project/Source Data\GPW%20Regional%20Tables%20-%20March%202024.xlsx
Processing URL: https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-april-2024
No suitable file found for URL: https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/30-april-2024
Processing URL: https://digital.nhs.uk/data-and-information/publications/statistical/general-and-personal-medical-services/29-february-2024
No suitable file found for URL: https://d

In [56]:
def clean_and_verify_data(df, column_name='COUNT_OF_APPOINTMENTS'):
    """
    Clean the specified column by removing asterisks and verify the cleaning process.
    
    Args:
    df (pd.DataFrame): The DataFrame to clean
    column_name (str): The name of the column to clean, default is 'count_of_appointments'
    
    Returns:
    pd.DataFrame: The cleaned DataFrame
    """
    initial_row_count = len(df)
    
    # Remove rows where the column contains an asterisk
    df_cleaned = df[df[column_name] != '*'].copy()
    
    rows_removed = initial_row_count - len(df_cleaned)
    
    print(f"Initial row count: {initial_row_count}")
    print(f"Rows removed: {rows_removed}")
    print(f"Remaining rows: {len(df_cleaned)}")
    
    return df_cleaned

# Usage
all_regional_data = clean_and_verify_data(all_regional_data)

# Double-check the 'count_of_appointments' column
asterisk_count = all_regional_data[all_regional_data['COUNT_OF_APPOINTMENTS'] == '*'].shape[0]
print(f"\nNumber of asterisks remaining in 'count_of_appointments': {asterisk_count}")

Initial row count: 1268228
Rows removed: 0
Remaining rows: 1268228

Number of asterisks remaining in 'count_of_appointments': 0


In [58]:
all_actual_duration_data

,SUB_ICB_LOCATION_NAME,ICB_ONS_CODE,REGION_ONS_CODE,Appointment_Date,ACTUAL_DURATION,COUNT_OF_APPOINTMENTS
0,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01DEC2021,1-5 Minutes,1539
1,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01DEC2021,31-60 Minutes,364
2,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01DEC2021,Unknown / Data Quality,1277
3,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01DEC2021,16-20 Minutes,730
4,NHS North East and North Cumbria ICB - 00L,E54000050,E40000012,01DEC2021,11-15 Minutes,1073
...,...,...,...,...,...,...
594016,NHS West Yorkshire ICB - X2C4Y,E54000054,E40000012,31MAY2024,31-60 Minutes,327
594017,NHS West Yorkshire ICB - X2C4Y,E54000054,E40000012,31MAY2024,21-30 Minutes,678
594018,NHS West Yorkshire ICB - X2C4Y,E54000054,E40000012,31MAY2024,16-20 Minutes,883
594019,NHS West Yorkshire ICB - X2C4Y,E54000054,E40000012,31MAY2024,11-15 Minutes,1446


In [12]:
def optimize_icb_name_assignment_regional(df):
    # Create a mapping dictionary from CCG to ICB New Name
    ccg_to_icb_map = dict(zip(df_ccg_icb['CCG'], df_ccg_icb['ICB New Name']))
    
    # Function to extract ICB name from SUB_ICB_LOCATION_NAME
    def extract_icb_from_sub_icb(name):
        if pd.isna(name):
            return np.nan
        parts = name.split('-')
        return parts[0].strip() if len(parts) > 1 else np.nan
    
    # Step 1: Keep existing ICB_NAME values
    icb_name = df['ICB_NAME'].copy()
    
    # Step 2: For rows with NaN ICB_NAME, try to assign based on CCG_NAME
    mask_nan_icb = icb_name.isna()
    ccg_names = df.loc[mask_nan_icb, 'CCG_NAME']
    icb_from_ccg = ccg_names.map(ccg_to_icb_map)
    icb_name.loc[mask_nan_icb] = icb_from_ccg
    
    # Step 3: For remaining NaN rows, try to extract from SUB_ICB_LOCATION_NAME
    mask_still_nan = icb_name.isna()
    sub_icb_names = df.loc[mask_still_nan, 'SUB_ICB_LOCATION_NAME']
    icb_from_sub_icb = sub_icb_names.apply(extract_icb_from_sub_icb)
    icb_name.loc[mask_still_nan] = icb_from_sub_icb
    
    return icb_name

def optimize_icb_name_assignment_actual_duration(df):
    # Function to extract ICB name from SUB_ICB_LOCATION_NAME
    def extract_icb_from_sub_icb(name):
        if pd.isna(name):
            return np.nan
        parts = name.split('-')
        return parts[0].strip() if len(parts) > 1 else np.nan
    
    # Extract ICB_NAME directly from SUB_ICB_LOCATION_NAME
    icb_name = df['SUB_ICB_LOCATION_NAME'].apply(extract_icb_from_sub_icb)
    
    return icb_name

# Apply the optimized functions to the respective dataframes
all_regional_data['ICB_NAME'] = optimize_icb_name_assignment_regional(all_regional_data)
all_actual_duration_data['ICB_NAME'] = optimize_icb_name_assignment_actual_duration(all_actual_duration_data)


# Display the updated DataFrames
print("Updated all_regional_data DataFrame:")
print(all_regional_data[['CCG_NAME', 'ICB_NAME', 'SUB_ICB_LOCATION_NAME']].head(10))
print("\nUpdated all_actual_duration_data DataFrame:")
print(all_actual_duration_data[['ICB_NAME', 'SUB_ICB_LOCATION_NAME']].head(10))

# Check remaining NaN values
regional_nan_counts = all_regional_data['ICB_NAME'].isna().sum()
duration_nan_counts = all_actual_duration_data['ICB_NAME'].isna().sum()
print("\nRemaining NaN values in ICB_NAME:")
print(f"Regional data: {regional_nan_counts}")
print(f"Actual duration data: {duration_nan_counts}")

# Check for rows where ICB_NAME is null but SUB_ICB_LOCATION_NAME is not
print("\nSample of rows where ICB_NAME is null but SUB_ICB_LOCATION_NAME is not (Regional data):")
print(all_regional_data[all_regional_data['ICB_NAME'].isna() & all_regional_data['SUB_ICB_LOCATION_NAME'].notna()].head())
print("\nSample of rows where ICB_NAME is null but SUB_ICB_LOCATION_NAME is not (Actual duration data):")
print(all_actual_duration_data[all_actual_duration_data['ICB_NAME'].isna() & all_actual_duration_data['SUB_ICB_LOCATION_NAME'].notna()].head())

Updated all_regional_data DataFrame:
  CCG_NAME                                  ICB_NAME  \
0      NaN  NHS West Yorkshire Integrated Care Board   
1      NaN  NHS West Yorkshire Integrated Care Board   
2      NaN  NHS West Yorkshire Integrated Care Board   
3      NaN  NHS West Yorkshire Integrated Care Board   
4      NaN  NHS West Yorkshire Integrated Care Board   
5      NaN  NHS West Yorkshire Integrated Care Board   
6      NaN  NHS West Yorkshire Integrated Care Board   
7      NaN  NHS West Yorkshire Integrated Care Board   
8      NaN  NHS West Yorkshire Integrated Care Board   
9      NaN  NHS West Yorkshire Integrated Care Board   

          SUB_ICB_LOCATION_NAME  
0  NHS West Yorkshire ICB - 02T  
1  NHS West Yorkshire ICB - 02T  
2  NHS West Yorkshire ICB - 02T  
3  NHS West Yorkshire ICB - 02T  
4  NHS West Yorkshire ICB - 02T  
5  NHS West Yorkshire ICB - 02T  
6  NHS West Yorkshire ICB - 02T  
7  NHS West Yorkshire ICB - 02T  
8  NHS West Yorkshire ICB - 02T  
9  NHS

In [64]:
# List of columns and number of NaN values in each column
regional_nan_counts = all_regional_data.isna().sum()
print(duration_nan_counts)
all_regional_data.info()


SUB_ICB_LOCATION_NAME    0
ICB_ONS_CODE             0
REGION_ONS_CODE          0
Appointment_Date         0
ACTUAL_DURATION          0
COUNT_OF_APPOINTMENTS    0
ICB_NAME                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268228 entries, 0 to 1268227
Data columns (total 12 columns):
 #   Column                      Non-Null Count    Dtype 
---  ------                      --------------    ----- 
 0   SUB_ICB_LOCATION_NAME       642124 non-null   object
 1   ICB_ONS_CODE                642124 non-null   object
 2   ICB_NAME                    1268228 non-null  object
 3   REGION_ONS_CODE             1268228 non-null  object
 4   REGION_NAME                 1268228 non-null  object
 5   APPOINTMENT_MONTH           1268228 non-null  object
 6   APPT_STATUS                 1268228 non-null  object
 7   HCP_TYPE                    1268228 non-null  object
 8   APPT_MODE                   1268228 non-null  object
 9   TIME_BETWEEN_BOOK_AND_APPT  1268228 non-nu

In [65]:
duration_nan_counts = all_actual_duration_data.isna().sum()
print(duration_nan_counts)
all_actual_duration_data.info()

SUB_ICB_LOCATION_NAME    0
ICB_ONS_CODE             0
REGION_ONS_CODE          0
Appointment_Date         0
ACTUAL_DURATION          0
COUNT_OF_APPOINTMENTS    0
ICB_NAME                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594021 entries, 0 to 594020
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   SUB_ICB_LOCATION_NAME  594021 non-null  object
 1   ICB_ONS_CODE           594021 non-null  object
 2   REGION_ONS_CODE        594021 non-null  object
 3   Appointment_Date       594021 non-null  object
 4   ACTUAL_DURATION        594021 non-null  object
 5   COUNT_OF_APPOINTMENTS  594021 non-null  int64 
 6   ICB_NAME               594021 non-null  object
dtypes: int64(1), object(6)
memory usage: 31.7+ MB


In [66]:
def extract_icb_from_sub_icb(name):
    if pd.isna(name):
        return name
    parts = name.split('-')
    if len(parts) > 1:
        return parts[0].strip()
    return name

def map_icb_names(all_actual_duration_data, all_regional_data):
    # Create ICB_ONS_CODE to ICB_NAME mapping from all_regional_data
    icb_code_to_name = all_regional_data.dropna(subset=['ICB_ONS_CODE', 'ICB_NAME']).drop_duplicates(subset=['ICB_ONS_CODE'])
    icb_code_to_name = dict(zip(icb_code_to_name['ICB_ONS_CODE'], icb_code_to_name['ICB_NAME']))
    
    # Step 1: Map using ICB_ONS_CODE
    all_actual_duration_data['ICB_NAME'] = all_actual_duration_data['ICB_ONS_CODE'].map(icb_code_to_name)
    
    # Step 2: For remaining NaNs, extract ICB name from SUB_ICB_LOCATION_NAME
    mask = all_actual_duration_data['ICB_NAME'].isna()
    all_actual_duration_data.loc[mask, 'ICB_NAME'] = all_actual_duration_data.loc[mask, 'SUB_ICB_LOCATION_NAME'].apply(extract_icb_from_sub_icb)
    
    return all_actual_duration_data

# Usage
all_actual_duration_data = map_icb_names(all_actual_duration_data, all_regional_data)

# Check remaining NaN values
print(f"Remaining NaN values in ICB_NAME: {all_actual_duration_data['ICB_NAME'].isna().sum()}")

# Display sample of mapped data
print(all_actual_duration_data[['SUB_ICB_LOCATION_NAME', 'ICB_NAME', 'ICB_ONS_CODE']].sample(10))

# Check unique values in ICB_NAME
print(f"Number of unique ICB_NAME values: {all_actual_duration_data['ICB_NAME'].nunique()}")
print(all_actual_duration_data['ICB_NAME'].value_counts())

Remaining NaN values in ICB_NAME: 0
                                    SUB_ICB_LOCATION_NAME  \
334192                      NHS Gloucestershire ICB - 11M   
68023                    NHS Greater Manchester ICB - 01G   
37433          NHS Lancashire and South Cumbria ICB - 00X   
278575                  NHS Mid and South Essex ICB - 06Q   
552142          NHS Coventry and Warwickshire ICB - B2M3M   
122043              NHS Cheshire and Merseyside ICB - 02E   
306419                               NHS Sussex ICB - 09D   
430286                       NHS West Yorkshire ICB - 36J   
570649  NHS Bedfordshire Luton and Milton Keynes ICB -...   
530855                  NHS Mid and South Essex ICB - 99F   

                                            ICB_NAME ICB_ONS_CODE  
334192                       NHS Gloucestershire ICB    E54000043  
68023                     NHS Greater Manchester ICB    E54000057  
37433           NHS Lancashire and South Cumbria ICB    E54000048  
278575              

In [67]:
#Add icb_name by dropping everything past '-' in ONS Name
combined_fte_df['icb_name'] = combined_fte_df['ONS Name'].str.split(' - ').str[0]

#Create FTE-GP by calculating
combined_fte_df['FTE-GP'] = (
    0.4 * combined_fte_df['<= 0.4 FTE'] +
    0.99 * combined_fte_df['> 0.4 to < 1 FTE'] +
    1.31 * combined_fte_df['>= 1 FTE']
)

# Create report_month by extracting from file name
def extract_report_month(filename):
    match = re.search(r'(\w+)%20(\d{4})', filename)
    if match:
        month, year = match.groups()
        month = re.sub(r'[^a-zA-Z]', '', month)
        try:
            date = datetime.strptime(f"1 {month} {year}", "%d %B %Y")
        except ValueError:
            try:
                date = datetime.strptime(f"1 {month[:3]} {year}", "%d %b %Y")
            except ValueError:
                return None
        return date
    return None

combined_fte_df['report_month'] = combined_fte_df['file'].apply(extract_report_month)

# Check for any rows where report_month couldn't be extracted
null_report_months = combined_fte_df[combined_fte_df['report_month'].isnull()]
if not null_report_months.empty:
    print("\nRows where report_month couldn't be extracted:")
    print(null_report_months[['ONS Code', 'ONS Name', 'file']])
else:
    print("\nReport month was successfully extracted for all rows.")

# Drop unnecessary columns
combined_fte_df = combined_fte_df.drop(columns=['ONS Code', 'ONS Name', 'file'])

# Display the data before aggregation
print("Original Data before Aggregation:\n", combined_fte_df)

# Sum all the duplicates up for each icb_name and month
summed_df = combined_fte_df.groupby(['icb_name', 'report_month']).agg({
    '<= 0.4 FTE': 'sum',
    '> 0.4 to < 1 FTE': 'sum',
    '>= 1 FTE': 'sum',
    'FTE-GP': 'sum'
}).reset_index()

# Display the data after aggregation
print("\nSummed Data after Aggregation:\n", summed_df)

# Define the extend_fte_data function
def extend_fte_data(fte, start_date, end_date):
    # Sort the fte DataFrame by report_month
    fte_sorted = fte.sort_values('report_month')
    
    # Create a DataFrame with all months in the specified range
    all_months = pd.DataFrame({'report_month': pd.date_range(start=start_date, end=end_date, freq='MS')})
    
    # Create a list to store the extended FTE data
    extended_fte_list = []
    
    # Iterate through the ICBs
    for icb in fte['icb_name'].unique():
        icb_fte = fte_sorted[fte_sorted['icb_name'] == icb].copy()
        icb_fte = icb_fte.set_index('report_month')
        
        # Extend the FTE data for this ICB
        extended_icb_fte = all_months.copy()
        extended_icb_fte['icb_name'] = icb
        extended_icb_fte = extended_icb_fte.set_index('report_month')
        
        # Merge and forward fill the FTE data
        # Use suffixes to avoid duplicate column names
        extended_icb_fte = pd.merge(extended_icb_fte, icb_fte, left_index=True, right_index=True, 
                                    how='left', suffixes=('', '_y'))
        
        # Drop the duplicate 'icb_name' column if it was created
        if 'icb_name_y' in extended_icb_fte.columns:
            extended_icb_fte = extended_icb_fte.drop(columns=['icb_name_y'])
        
        extended_icb_fte = extended_icb_fte.ffill()
        
        extended_fte_list.append(extended_icb_fte.reset_index())
    
    # Combine all extended FTE data
    extended_fte = pd.concat(extended_fte_list, ignore_index=True)
    
    return extended_fte

# Step 6: Extend the FTE data for each ICB and forward fill missing values
start_date = '2022-09-01'
end_date = '2024-07-01'
extended_fte_df = extend_fte_data(summed_df, start_date, end_date)

# Display the final dataframe
print("\nExtended FTE Data:\n", extended_fte_df)


Report month was successfully extracted for all rows.
Original Data before Aggregation:
      <= 0.4 FTE  > 0.4 to < 1 FTE  >= 1 FTE                      icb_name  \
0         121.0             716.0     218.0     NHS South East London ICB   
1         117.0             709.0     305.0     NHS North East London ICB   
2         114.0             691.0     187.0  NHS North Central London ICB   
3         159.0             811.0     336.0     NHS North West London ICB   
4         105.0             737.0     238.0     NHS South West London ICB   
..          ...               ...       ...                           ...   
419        10.0              82.0      14.0        NHS West Yorkshire ICB   
420         7.0             170.0      52.0        NHS West Yorkshire ICB   
421        30.0             479.0      85.0        NHS West Yorkshire ICB   
422        30.0             316.0      85.0        NHS West Yorkshire ICB   
423        13.0             130.0      77.0        NHS West Yor

In [ ]:
def check_duplicates(df):
    # Check for duplicates based on 'icb_name' and 'report_month'
    duplicates = df[df.duplicated(subset=['icb_name', 'report_month'], keep=False)]
    
    if duplicates.empty:
        print("No duplicates found. Each ICB has only one entry per month.")
    else:
        print("Duplicates found:")
        print(duplicates.sort_values(['icb_name', 'report_month']))
        
        # Count the number of duplicates for each ICB and month
        duplicate_counts = duplicates.groupby(['icb_name', 'report_month']).size().reset_index(name='count')
        print("\nDuplicate counts:")
        print(duplicate_counts)
        
    # Additional check: verify that each ICB has an entry for every month
    all_months = df['report_month'].unique()
    all_icbs = df['icb_name'].unique()
    
    expected_count = len(all_months)
    
    icb_month_counts = df.groupby('icb_name').size()
    icbs_with_missing_months = icb_month_counts[icb_month_counts < expected_count]
    
    if icbs_with_missing_months.empty:
        print("\nAll ICBs have an entry for every month.")
    else:
        print("\nICBs with missing months:")
        print(icbs_with_missing_months)

# Assuming extended_fte_df is your processed DataFrame
check_duplicates(extended_fte_df)

# Optional: Display some summary statistics
print("\nSummary statistics:")
print(extended_fte_df.describe())

# Optional: Display the first few rows of the DataFrame
print("\nFirst few rows of the DataFrame:")
print(extended_fte_df.head())

# Optional: Display the shape of the DataFrame
print(f"\nShape of the DataFrame: {extended_fte_df.shape}")

No duplicates found. Each ICB has only one entry per month.

All ICBs have an entry for every month.

Summary statistics:
                        report_month  <= 0.4 FTE  > 0.4 to < 1 FTE  \
count                            966  966.000000        966.000000   
mean   2023-08-01 08:20:52.173913088   70.860248        589.167702   
min              2022-09-01 00:00:00   14.000000        212.000000   
25%              2023-02-01 00:00:00   39.000000        366.000000   
50%              2023-08-01 00:00:00   62.000000        529.000000   
75%              2024-02-01 00:00:00  102.000000        719.000000   
max              2024-07-01 00:00:00  169.000000       1522.000000   
std                              NaN   37.745729        288.074916   

         >= 1 FTE      FTE-GP  
count  966.000000   966.00000  
mean   185.650104   854.82176  
min     68.000000   312.42000  
25%    112.000000   563.34000  
50%    174.000000   734.01000  
75%    233.750000  1063.48000  
max    454.000000  1981

In [68]:
def update_icb_name(name):
    mapping = {
        "NHS Bristol, North Somerset and South Gloucestershire ICB": "NHS Bristol North Somerset and South Gloucestershire ICB",
        "NHS Bath and North East Somerset, Swindon and Wiltshire ICB": "NHS Bath and North East Somerset Swindon and Wiltshire ICB",
        "NHS Staffordshire and Stoke": "NHS Staffordshire and Stoke-on-Trent ICB",
        "NHS Leicester, Leicestershire and Rutland ICB": "NHS Leicester Leicestershire and Rutland ICB",
        "NHS Buckinghamshire, Oxfordshire and Berkshire West ICB": "NHS Buckinghamshire Oxfordshire and Berkshire West ICB",
        "NHS Cornwall and The Isles Of Scilly ICB": "NHS Cornwall and The Isles of Scilly ICB",
        "NHS Bedfordshire, Luton and Milton Keynes ICB": "NHS Bedfordshire Luton and Milton Keynes ICB",
        "NHS Hampshire and Isle Of Wight ICB": "NHS Hampshire and the Isle of Wight ICB",
        "NHS Shropshire, Telford and Wrekin ICB": "NHS Shropshire Telford and Wrekin ICB"
    }
    return mapping.get(name, name)

# Apply the update to the ICB_NAME column
combined_fte_df['icb_name'] = combined_fte_df['icb_name'].apply(update_icb_name)

# Verify the changes
print("Updated ICB Names in FTE data:")
print(combined_fte_df['icb_name'].value_counts())

# Compare with regional data again
fte_icb_names = set(combined_fte_df['icb_name'].unique())
regional_icb_names = set(all_regional_data['ICB_NAME'].unique())

# Find any remaining differences
fte_only_icb = fte_icb_names - regional_icb_names
regional_only_icb = regional_icb_names - fte_icb_names

print("\nRemaining ICB names in FTE data but not in regional data:")
for icb in fte_only_icb:
    print(f"- {icb}")

print("\nRemaining ICB names in regional data but not in FTE data:")
for icb in regional_only_icb:
    print(f"- {icb}")

# Display statistics
print(f"\nTotal unique ICB names in updated FTE data: {len(fte_icb_names)}")
print(f"Total unique ICB names in regional data: {len(regional_icb_names)}")
print(f"Number of matching ICB names after update: {len(fte_icb_names.intersection(regional_icb_names))}")

# Check for any remaining discrepancies
if fte_only_icb or regional_only_icb:
    print("\nThere are still some discrepancies between FTE and regional ICB names.")
    print("You may want to review these manually.")
else:
    print("\nAll ICB names in FTE data now match with regional data.")

# Optionally, display a sample of the updated dataframe
print("\nSample of updated FTE data:")
print(combined_fte_df[['icb_name']].head())

Updated ICB Names in FTE data:
icb_name
NHS Greater Manchester ICB                                    40
NHS Cheshire and Merseyside ICB                               36
NHS North East and North Cumbria ICB                          32
NHS Lancashire and South Cumbria ICB                          32
NHS Staffordshire and Stoke-on-Trent ICB                      24
NHS Humber and North Yorkshire ICB                            24
NHS West Yorkshire ICB                                        20
NHS Mid and South Essex ICB                                   20
NHS South Yorkshire ICB                                       16
NHS Buckinghamshire Oxfordshire and Berkshire West ICB        12
NHS Sussex ICB                                                12
NHS Suffolk and North East Essex ICB                          12
NHS Hertfordshire and West Essex ICB                          12
NHS Leicester Leicestershire and Rutland ICB                  12
NHS Hampshire and the Isle of Wight ICB           

In [69]:
extended_fte_df

,report_month,icb_name,<= 0.4 FTE,> 0.4 to < 1 FTE,>= 1 FTE,FTE-GP
0,2022-09-01,"NHS Bath and North East Somerset, Swindon and ...",70.0,452.0,105.0,613.03
1,2022-10-01,"NHS Bath and North East Somerset, Swindon and ...",70.0,452.0,105.0,613.03
2,2022-11-01,"NHS Bath and North East Somerset, Swindon and ...",70.0,452.0,105.0,613.03
3,2022-12-01,"NHS Bath and North East Somerset, Swindon and ...",70.0,452.0,105.0,613.03
4,2023-01-01,"NHS Bath and North East Somerset, Swindon and ...",70.0,452.0,105.0,613.03
...,...,...,...,...,...,...
961,2024-03-01,NHS West Yorkshire ICB,92.0,1216.0,291.0,1621.85
962,2024-04-01,NHS West Yorkshire ICB,92.0,1216.0,291.0,1621.85
963,2024-05-01,NHS West Yorkshire ICB,92.0,1216.0,291.0,1621.85
964,2024-06-01,NHS West Yorkshire ICB,92.0,1216.0,291.0,1621.85


In [70]:
duplicates = extended_fte_df[extended_fte_df.duplicated(subset=['icb_name', 'report_month'], keep=False)]
if duplicates.empty:
    print("No duplicates found.")
else:
    print("Duplicates found:")
    print(duplicates)

No duplicates found.


In [71]:
def create_schema_if_not_exists(engine, schema):
    inspector = inspect(engine)
    if schema not in inspector.get_schema_names():
        with engine.begin() as conn:
            conn.execute(CreateSchema(schema))
        print(f"Created schema: {schema}")
    else:
        print(f"Schema {schema} already exists")

def drop_table_if_exists(engine, table_name, schema='raw'):
    inspector = inspect(engine)
    if inspector.has_table(table_name, schema=schema):
        with engine.begin() as conn:
            conn.execute(text(f"DROP TABLE IF EXISTS {schema}.{table_name}"))
        print(f"Dropped existing table {schema}.{table_name}")
    else:
        print(f"Table {schema}.{table_name} does not exist")

def safe_to_sql(df, table_name, engine, schema='raw', delete_existing=False, **kwargs):
    if isinstance(df, pd.DataFrame):
        try:
            create_schema_if_not_exists(engine, schema)
            if delete_existing:
                drop_table_if_exists(engine, table_name, schema)
            df.to_sql(table_name, engine, schema=schema, **kwargs)
            print(f"Successfully uploaded {table_name} to the {schema} schema in the database.")
        except Exception as e:
            print(f"Error uploading {table_name}: {str(e)}")
    else:
        print(f"Error: {table_name} is not a DataFrame. Type: {type(df)}")

def chunked_to_sql(df, table_name, engine, schema='raw', delete_existing=False, chunksize=100000, **kwargs):
    if isinstance(df, pd.DataFrame):
        try:
            create_schema_if_not_exists(engine, schema)
            if delete_existing:
                drop_table_if_exists(engine, table_name, schema)
            total_rows = len(df)
            chunks = range(0, total_rows, chunksize)
            for start_i in chunks:
                end_i = start_i + chunksize
                if_exists = 'replace' if start_i == 0 else 'append'
                df.iloc[start_i:end_i].to_sql(table_name, engine, schema=schema, if_exists=if_exists, index=False, **kwargs)
                print(f"Uploaded chunk {start_i}-{end_i} of {total_rows} rows to {schema}.{table_name}")
        except Exception as e:
            print(f"Error uploading {table_name}: {str(e)}")
    else:
        print(f"Error: {table_name} is not a DataFrame. Type: {type(df)}")

# Convert all column names to lowercase
national_categories_data.columns = national_categories_data.columns.str.lower()
extended_fte_df.columns = extended_fte_df.columns.str.lower()  # Corrected from extend_fte_data to extended_fte_df
all_actual_duration_data.columns = all_actual_duration_data.columns.str.lower()
all_regional_data.columns = all_regional_data.columns.str.lower()

# Drop unnecessary columns
national_categories_data = national_categories_data.drop(columns=['sub_icb_location_code', 'sub_icb_location_name', 'sub_icb_location_ons_code', 'icb_ons_code', 'region_ons_code'])

# Corrected the column dropping for these DataFrames
columns_to_drop = ['sub_icb_location_name', 'icb_ons_code', 'region_ons_code', 'ccg_name']
all_actual_duration_data = all_actual_duration_data.drop(columns=[col for col in columns_to_drop if col in all_actual_duration_data.columns])
all_regional_data = all_regional_data.drop(columns=[col for col in columns_to_drop if col in all_regional_data.columns])


# Usage
# For smaller DataFrames
safe_to_sql(national_categories_data, "national_categories_data", engine, if_exists="replace", index=False, delete_existing=True)
safe_to_sql(extended_fte_df, "combined_fte_df", engine, if_exists="replace", index=False, delete_existing=True)
safe_to_sql(df_ccg_icb, "df_ccg_icb", engine, if_exists="replace", index=False, delete_existing=True)

# For larger DataFrames
chunked_to_sql(all_actual_duration_data, "all_actual_duration_data", engine, delete_existing=True)
chunked_to_sql(all_regional_data, "all_regional_data", engine, delete_existing=True)

Schema raw already exists
Dropped existing table raw.national_categories_data
Successfully uploaded national_categories_data to the raw schema in the database.
Schema raw already exists
Dropped existing table raw.combined_fte_df
Successfully uploaded combined_fte_df to the raw schema in the database.
Schema raw already exists
Dropped existing table raw.df_ccg_icb
Successfully uploaded df_ccg_icb to the raw schema in the database.
Schema raw already exists
Dropped existing table raw.all_actual_duration_data
Uploaded chunk 0-100000 of 594021 rows to raw.all_actual_duration_data
Uploaded chunk 100000-200000 of 594021 rows to raw.all_actual_duration_data
Uploaded chunk 200000-300000 of 594021 rows to raw.all_actual_duration_data
Uploaded chunk 300000-400000 of 594021 rows to raw.all_actual_duration_data
Uploaded chunk 400000-500000 of 594021 rows to raw.all_actual_duration_data
Uploaded chunk 500000-600000 of 594021 rows to raw.all_actual_duration_data
Schema raw already exists
Dropped exi